# 第一章：初探大模型智能体

## 1. 背景与意义

智能体技术的发展**降低了使用AI的门槛**，使得更多人能够便捷地利用AI能力。

### 技术演进对比

- **ChatGPT**：主要处理**文本交互**，通过对话形式完成问答任务
- **AutoGPT**：*"Let AI work for you, and have your ideas come to life"*
  - 技术上没有根本性变化
  - **核心变革在于交互范式的转变**

---

## 2. 什么是智能体？

### 2.1 强化学习的定义

智能体的经典定义遵循三个核心环节：

1. **感知** (Perception)
2. **思考** (Thinking)  
3. **行动** (Action)

流程：**感知 → 思考 → 行动**

### 2.2 大模型智能体的特点

- **碎片化情景用语言空间描述**：将复杂环境抽象为语言表示
- **以LLM为中心**：大型语言模型作为核心决策引擎
- **配合工具调用执行决策**：通过外部工具扩展能力边界
- **本质不变**：仍然是**感知 → 思考 → 行动**的基本框架

### 2.3 智能体的交互方式

智能体存在两类主要交互模式：

1. **智能体与人的交互**
   - 用户提供指令
   - 智能体理解并执行
   - 返回结果给用户

2. **智能体与环境的交互**
   - 感知环境状态
   - 根据环境反馈调整策略
   - 执行动作改变环境

### 2.4 系统组成要素

智能体系统的核心要素包括：
- **用户**：指令的发起者
- **智能体**：决策与执行的核心
- **记忆**：存储历史信息与状态

---

## 3. 核心价值

**新的交互范式**使得人们能够：
- 更加**简洁**地表达需求
- 更加**高效**地获得所需功能
- 降低技术使用门槛，提升生产力




---

## 4. 实践代码

### 4.1 环境准备

**Step 1: 安装依赖并导入库**

In [12]:
# 安装必要的依赖库
%pip install openai python-dotenv

# 导入必要的库
import json
import os
from pathlib import Path
from openai import OpenAI

Note: you may need to restart the kernel to use updated packages.


### 4.2 配置文件管理

**Step 2: 从JSON配置文件读取API密钥**

> 💡 **安全提示**：`config.json` 文件已添加到 `.gitignore`，不会被上传到GitHub。请确保创建并填写 `config.json` 文件。

In [13]:
def load_config(config_path=None):
    """
    从JSON文件加载API配置
    自动查找配置文件：优先查找 HandsOn/config.json
    
    Args:
        config_path: 配置文件路径，如果为None则自动查找
    
    Returns:
        dict: 包含所有API配置的字典，如果失败返回None
    """
    # 如果未指定路径，自动查找配置文件
    if config_path is None:
        # 可能的配置文件位置（按优先级排序）
        possible_paths = [
            "config.json",  # 当前目录（如果notebook在HandsOn目录运行）
            "HandsOn/config.json",  # HandsOn子目录
            "../HandsOn/config.json",  # 上级目录的HandsOn文件夹
            os.path.join(os.getcwd(), "HandsOn", "config.json"),  # 绝对路径
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                config_path = path
                break
        
        if config_path is None:
            print("❌ 错误: 找不到配置文件")
            print("   请确保在 HandsOn/ 目录下存在 config.json 文件")
            print("   可以从 HandsOn/config.json.example 复制并填写你的API密钥")
            return None
    
    try:
        # 转换为绝对路径并读取
        config_path = os.path.abspath(config_path)
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)
        print(f"✅ 成功加载配置文件: {config_path}")
        return config
    except FileNotFoundError:
        print(f"❌ 错误: 找不到配置文件 {config_path}")
        print("   请复制 HandsOn/config.json.example 为 HandsOn/config.json 并填写你的API密钥")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ 错误: JSON格式错误 - {e}")
        print(f"   请检查 {config_path} 文件的JSON语法")
        return None

# 加载配置（自动查找 HandsOn/config.json）
config = load_config()

# 查看已配置的API平台（不显示密钥）
if config:
    print("\n📋 已配置的API平台:")
    for platform in config.keys():
        model_name = config[platform].get('model', 'N/A')
        base_url = config[platform].get('base_url', 'N/A')
        has_key = bool(config[platform].get('api_key'))
        status = "✅" if has_key else "⚠️"
        print(f"  {status} {platform.upper()}: {model_name} @ {base_url}")

✅ 成功加载配置文件: c:\Users\Peiyan Li\Desktop\Hands-on-aai\HandsOn\config.json

📋 已配置的API平台:
  ✅ OPENAI: gpt-4o-mini @ https://api.openai.com/v1
  ✅ CHATGLM: GLM-4.7 @ https://open.bigmodel.cn/api/coding/paas/v4


### 4.3 定义智能体类

**Step 3: 创建问答智能体类（支持从配置文件读取）**

In [14]:
class QAgent:
    """
    问答智能体类，支持从配置文件读取API信息
    兼容 OpenAI API 格式（OpenAI、ChatGLM等）
    """
    def __init__(self, api_key=None, base_url=None, model=None, 
                 platform="openai", config=None, 
                 system_prompt="你是一个很聪明的智能体。你会用中文回答用户提出的任何问题。"):
        """
        初始化智能体
        
        Args:
            api_key: API密钥（可选，优先使用直接提供的参数）
            base_url: API基础URL（可选，优先使用直接提供的参数）
            model: 模型名称（可选，优先使用直接提供的参数）
            platform: 平台名称，用于从config中读取配置（可选: "openai", "chatglm"）
            config: 配置字典（可选，如果不提供则自动从文件加载）
            system_prompt: 系统提示词，定义智能体角色和行为
        """
        # 如果提供了config参数，使用它；否则从文件加载
        if config is None:
            config_path = "config.json" if os.path.exists("config.json") else "HandsOn/config.json"
            config = load_config(config_path)
        
        # 优先使用直接提供的参数，否则从config中读取
        if config:
            platform_config = config.get(platform, {})
            self.api_key = api_key or platform_config.get("api_key")
            self.base_url = base_url or platform_config.get("base_url")
            self.model = model or platform_config.get("model")
        else:
            # 如果没有config，必须提供参数
            if not all([api_key, base_url, model]):
                raise ValueError(
                    "❌ 错误: 配置文件不可用且未提供完整参数。\n"
                    "   请提供 api_key、base_url 和 model 参数，"
                    "   或创建 HandsOn/config.json 配置文件"
                )
            self.api_key = api_key
            self.base_url = base_url
            self.model = model
        
        # 验证必要参数
        if not self.api_key or not self.base_url or not self.model:
            raise ValueError(
                f"❌ 错误: 未找到 {platform} 的完整API配置。\n"
                f"   请检查 HandsOn/config.json 文件，确保包含以下字段：\n"
                f"   - api_key\n"
                f"   - base_url\n"
                f"   - model"
            )
        
        self.platform = platform
        self.system_prompt = system_prompt
        self.client = OpenAI(api_key=self.api_key, base_url=self.base_url)
        
        print(f"✅ 智能体初始化成功")
        print(f"   平台: {platform.upper()}")
        print(f"   模型: {self.model}")
        print(f"   API地址: {self.base_url}")

    def ask(self, question, temperature=0.7, max_tokens=2048):
        """
        提问接口
        
        Args:
            question: 用户问题
            temperature: 温度参数（控制随机性，0-1之间）
            max_tokens: 最大生成token数
        
        Returns:
            str: 智能体的回答
        """
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": question},
                ],
                temperature=temperature,
                max_tokens=max_tokens,
                stream=False
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"❌ 错误: {str(e)}"
    
    def update_system_prompt(self, system_prompt):
        """更新系统提示词"""
        self.system_prompt = system_prompt
        print("✅ 系统提示词已更新")

### 4.4 实例化并使用智能体

**Step 4: 使用配置文件创建智能体实例**

In [15]:
# 方式1: 使用配置文件（推荐）- 使用OpenAI
agent = QAgent(platform="chatglm", config=config)

# 方式2: 使用ChatGLM平台（如果已配置）
# agent = QAgent(platform="chatglm", config=config)

# 方式3: 直接指定参数（如果需要覆盖配置文件中的设置）
# agent = QAgent(
#     api_key="your-api-key-here",
#     base_url="https://api.openai.com/v1",
#     model="gpt-4"
# )

✅ 智能体初始化成功
   平台: CHATGLM
   模型: GLM-4.7
   API地址: https://open.bigmodel.cn/api/coding/paas/v4


**Step 5: 使用智能体提问**

In [16]:
# 提出疑问并获取回复
question = "什么是大模型智能体？"
answer = agent.ask(question)
print(f"问题: {question}")
print(f"回答: {answer}")

问题: 什么是大模型智能体？
回答: **大模型智能体（LLM Agent）**，简单来说，就是**以大语言模型（LLM）为“大脑”，具备感知、规划、记忆和使用工具能力，能够自主完成复杂任务的智能系统。**

如果说大模型（比如 ChatGPT、Claude）是一个博学多才但只能“动口不动手”的智者，那么**大模型智能体就是给这位智者装上了“眼睛”、“耳朵”和“双手”，让它不仅能思考，还能真正地去“干活”。**

为了让你更透彻地理解，我们可以从以下几个维度来拆解：

### 1. 核心公式
业界通常用这个公式来定义智能体：
$$ \text{Agent} = \text{LLM (大模型)} + \text{Planning (规划)} + \text{Memory (记忆)} + \text{Tools (工具使用)} $$

### 2. 四大关键能力

一个优秀的智能体通常具备以下四个核心要素：

*   **大脑：大模型 (LLM)**
    *   这是基础，负责理解指令、逻辑推理、生成内容和决策。没有强大的大模型，智能体就无法理解复杂的需求。
*   **感知与规划**
    *   智能体不是像传统的程序那样死板地执行 `if-then` 代码。当你给它一个模糊的大目标（例如：“帮我策划一次去日本的旅行”），它会自己思考：“为了完成这个目标，我需要先查机票，再查酒店，然后做攻略，最后预算。”这种将大目标拆解为小步骤的能力就是规划。
*   **记忆**
    *   **短期记忆**：在当前对话中记住刚才说了什么。
    *   **长期记忆**：能够记住用户的偏好、历史操作经验，甚至从过去的错误中学习。例如，它记住了你不喜欢坐靠窗的位置，下次预订时会自动避开。
*   **工具使用**
    *   这是智能体区别于普通聊天机器人的关键。智能体可以调用外部 API 或软件来“动手”。比如：
        *   用 Google 搜索实时信息；
        *   用 Python 解释器写代码并运行；
        *   发送邮件、操作日程表、控制智能家居等。

### 3. 智能体 vs. 普通聊天机器人

| 特性 | 普通聊天机器人 (Chatbot) | 大模型智能体 |
| :--- | :--- | :--- |


### 4.5 美化输出显示

**Step 6: 使用 Markdown 格式美化输出**

> 提示：对于复杂的输出内容，大部分大模型会生成 Markdown 格式的输出，可以使用 IPython 的 Markdown 显示功能美化展示。

In [17]:
# 使用 IPython 的 Markdown 显示功能美化输出
# 注意：IPython.display.Markdown 是 IPython 自带的，不需要额外安装 markdown 包
from IPython.display import Markdown

# 重新获取回答并美化显示
answer = agent.ask("什么是大模型智能体？")
display(Markdown(answer))

**大模型智能体**（Large Language Model Agent，简称 **LLM Agent**）是一种基于大语言模型（如 GPT-4、Claude、Llama 等）构建的智能系统。

简单来说，如果说大模型是“**大脑**”，那么大模型智能体就是给这个大脑装上了“**感官**”、“**手脚**”和“**记忆**”，让它不仅能“说话”，还能“**办事**”。

以下从四个维度为你通俗易懂地解释：

### 1. 核心概念：从“被动回答”到“主动行动”

*   **传统大模型（聊天机器人）：**
    你问它答。它只能处理文本输入，生成文本输出。如果你问“今天北京天气怎么样？”，它可能会编造一个答案，或者告诉你它无法联网（取决于版本）。
*   **大模型智能体：**
    你给它一个**目标**（例如：“帮我查查北京天气，如果下雨就提醒我带伞，并把这周的天气做成表格发给我”）。它会**自主规划**步骤：先去联网查天气 -> 判断是否下雨 -> 如果是，执行“提醒”动作 -> 制作表格 -> 发送给你。

**一句话总结：智能体 = 大模型（大脑） + 感知 + 规划 + 工具使用 + 记忆。**

---

### 2. 大模型智能体的四大核心组件

一个完整的智能体通常包含以下几个部分：

1.  **大脑 - 规划与推理：**
    这是 LLM 的核心职责。它负责理解用户的复杂指令，将大目标拆解为一个个小任务（思维链 CoT），并决定下一步该做什么。
2.  **感知 - 眼睛和耳朵：**
    让智能体能“看”图片（多模态能力）、“听”语音，或者读取文件、网页内容，而不仅仅是处理文字。
3.  **工具 - 手和脚：**
    这是智能体最强大的地方。通过 **API (应用程序接口)**，大模型可以调用外部工具来改变现实世界或获取实时信息。例如：
    *   **搜索工具**（Google Search）：获取最新资讯。
    *   **代码解释器**（Python）：进行数据分析、画图、运行程序。
    *   **办公软件**（Calendar, Email）：自动安排日程、发邮件。
4.  **记忆 - 短期与长期记忆：**
    *   **短期记忆：** 记住刚才对话的上下文。
    *   **长期记忆：** 通过向量数据库等技术，记住用户的偏好（比如“我不吃香菜”）、历史操作经验，从而越用越聪明。

---

### 3. 工作流程举例

假设你命令智能体：“**帮我策划一次去日本的旅行。**”

一个智能体的工作流是这样的：
1.  **理解与规划：** LLM 分析需求，决定先询问你的预算和时间。
2.  **交互：** 它问你：“你的预算是多少？计划去几天？”
3.  **获取信息：** 你回答后，它调用“搜索工具”查找当下的机票价格和热门景点。
4.  **思考与决策：** 根据搜索结果，它发现A景点在装修，于是自动替换为B景点。
5.  **执行工具：** 它调用“文档工具”生成一份 PDF 行程单，或者直接在你的日历上创建行程提醒。

---

### 4. 为什么这很重要？

大模型智能体被认为是通往 **AGI（通用人工智能）** 的关键一步。

*   **从“玩具”变“工具”：** 以前大模型像是一个知识渊博但只能动嘴的学者；现在有了智能体架构，它变成了一位能帮你实际干活、解决复杂问题的全能助理。
*   **自主性：** 它不需要人类对每一个微小步骤都发出指令，可以自主地通过环境反馈来调整自己的行为。

**常见的例子：**
*   **Devin**：被誉为首个AI软件工程师，能独立写代码、修Bug、部署网站。
*   **AutoGPT**：一个开源项目，你给它一个目标（比如“帮我开发一个网页游戏”），它会自动循环生成提示词、执行代码、自我修正，直到完成。
*   **Office Copilot**：能帮你分析Excel数据，并直接生成PPT。

总而言之，**大模型智能体就是让 AI 从“能言善辩”进化为“知行合一”的系统。**

### 4.6 体验不同角色

**Step 7: 更新系统提示词，改变智能体角色**

通过更新系统提示词，可以让智能体扮演不同的角色，适应不同的使用场景。

In [18]:
# 更新系统提示词，改变智能体角色
agent.update_system_prompt("你是经验丰富的旅行家，你会用你的经验帮用户制定旅行攻略。")

# 提出新问题
new_question = "国庆节我想去上海，请问有什么游玩建议或美食推荐吗？"
new_answer = agent.ask(new_question)
display(Markdown(f"**问题**: {new_question}\n\n**回答**:\n{new_answer}"))

✅ 系统提示词已更新


**问题**: 国庆节我想去上海，请问有什么游玩建议或美食推荐吗？

**回答**:
太棒了！作为一名经验丰富的旅行家，我非常乐意为你规划上海之旅。但首先我要给你打一剂“预防针”：**国庆节期间的上海是真正的“人气之都”，人流量极大。**

因此，这份攻略的核心不仅仅是“去哪里玩”，更是**“如何在人多的情况下玩得舒服”**。

以下是为您量身定制的上海国庆游玩建议与美食推荐：

---

### 一、 🚦 出行前的“保命”建议（非常重要）

1.  **住宿策略：**
    *   **避开外滩/南京东路：** 除非预算充足且不介意噪音，否则不要住在核心景区。
    *   **首选地铁沿线：** 推荐**2号线**（串联虹桥机场/火车站、静安寺、陆家嘴、张江、浦东机场）或**10号线**（串联新天地、豫园、南京路、外滩、虹桥枢纽）。
    *   **推荐区域：** 静安寺附近（既繁华又方便）、陕西南路（法租界风情）、或者是人民广场附近（交通枢纽）。

2.  **门票预约（现在就要看！）：**
    *   上海几乎所有热门景点（迪士尼、博物馆、美术馆、登高建筑）都实行**实名制分时段预约**。
    *   *特备提醒：* 上海博物馆东馆、上海天文馆、迪士尼，国庆的票通常提前一周甚至更早就没了，**请立刻、马上查票！**

3.  **交通：**
    *   **放弃打车：** 国庆期间在市中心打车可能比走路还慢，且容易堵在隧道里。
    *   **地铁 + 共享单车：** 是最靠谱的组合。如果一定要过江，记得体验一下**轮渡**（比如东金线），只要2块钱，还能看江景，比挤地铁舒服。

---

### 二、 🏙️ 游玩建议：错峰与多元体验

为了避免人挤人，建议将行程分为“经典打卡”和“城市漫步”。

#### 1. 经典必打卡（建议清晨或深夜）
*   **外滩：**
    *   *建议：* 千万不要晚上7-9点去挤观景台。
    *   *最佳方案：* **早上6:30-7:30去**。晨光洒在万国建筑群上，人最少，拍照最出片，拍完正好去吃早茶。
*   **陆家嘴三件套（上海中心、环球金融、金茂）：**
    *   *建议：* 如果你想登高，推荐**上海中心大厦（朵云书院）**。虽然也要预约，但在52楼的高空书店看书喝茶，视野绝佳且逼格满满。
*   **豫园 & 城隍庙：**
    *   *建议：* 国庆期间这里会是“人从众”。如果是为了看风景，晚上亮灯后在外围转转即可；如果是为了吃小吃，没必要进去挤，周边有很多老字号。

#### 2. 城市漫步City Walk（上海特色）
上海最有味道的是街道，而不是景点。
*   **武康路 - 安福路路线：**
    *   打卡武康大楼（宋庆龄故居旁）。这里有最美的梧桐树和众多的咖啡店、买手店。虽然国庆人多，但街道空间大，散心很棒。
*   **多伦路 - 山阴路：**
    *   更有文化气息，鲁迅故居、茅盾故居都在这，保留了更多老上海的石库门味道。

#### 3. 艺术与

### 4.7 切换不同API平台

**Step 8: 尝试使用不同的API平台**

> 提示：需要先在 `HandsOn/config.json` 中配置对应平台的 API 信息

In [19]:
# 使用 ChatGLM 平台（如果已在 config.json 中配置）
agent_chatglm = QAgent(platform="chatglm", config=config)
answer_chatglm = agent_chatglm.ask("你好，介绍一下ChatGLM模型")
display(Markdown(answer_chatglm))

# 使用 OpenAI 平台（如果已在 config.json 中配置）
agent_openai = QAgent(platform="openai", config=config)
answer_openai = agent_openai.ask("你好，介绍一下GPT-4omini模型")
display(Markdown(answer_openai))

# 对比不同平台的回答风格
print("ChatGLM回答:", answer_chatglm)
print("OpenAI回答:", answer_openai)

✅ 智能体初始化成功
   平台: CHATGLM
   模型: GLM-4.7
   API地址: https://open.bigmodel.cn/api/coding/paas/v4


你好！我是由智谱AI研发的GLM大语言模型，很高兴为你介绍我自己以及ChatGLM系列模型。

**ChatGLM** 是由**清华大学 KEG 实验室**和**智谱 AI**（Zhipu AI）共同研发的、支持中英双语的对话语言模型系列。它基于 GLM（General Language Model）架构，旨在让机器能够像人类一样通过自然语言进行流畅的交互。

以下是关于 ChatGLM 模型的详细介绍，主要包含其架构特点、核心优势以及发展历程：

### 1. 核心架构：GLM (General Language Model)
不同于早期的 BERT（擅长理解但不擅长生成）或 GPT（擅长生成但使用自回归单向注意机制），ChatGLM 采用了 **GLM 架构**。
*   **自回归填空**：GLM 通过一种“自回归填空”的任务进行预训练，这使得它结合了双向编码器（理解能力强）和自回归解码器（生成能力强）的优点。
*   **优化部署**：ChatGLM 针对中文语境进行了深度的优化，并且针对推理速度和显存占用进行了特别的工程优化。

### 2. 核心优势
*   **出色的中文能力**：相比许多直接翻译中文提示词的国外模型，ChatGLM 在中文语义理解、成语运用、文化背景理解上表现更加地道和流畅。
*   **低部署门槛（消费级显卡可用）**：这是 ChatGLM 一开始就引爆开源社区的重要原因。通过 INT4 量化技术，ChatGLM-6B 版本可以在仅有 6GB 显存的消费级显卡（如家用游戏显卡）上流畅运行，极大地降低了个人开发者和中小企业使用大模型的门槛。
*   **开源生态**：智谱 AI 开放了多个版本的模型权重，极大地促进了中文大模型社区的发展，衍生出了许多微调版本和应用。
*   **长上下文与工具调用**：随着版本迭代，ChatGLM 系列在长文本处理、代码生成、网络搜索工具调用以及多模态（看图）能力上都有了显著提升。

### 3. 发展历程（主要版本）
ChatGLM 系列迭代非常迅速，以下是几个关键节点：

*   **ChatGLM-6B (初代)**：
    *   于 2022 年合作推出，2023 年开源。
    *   60 亿参数，针对中文问答和对话进行了优化。
    *   特点是“平民化”，让大家在家里的电脑上就能跑大模型。

*   **ChatGLM2-6B**：
    *   升级了基座模型，性能大幅提升。
    *   上下文长度扩展到 32K（能处理更长的文章）。
    *   采用了更先进的 Multi-Query Attention 技术，推理速度更快，显存占用更低。

*   **ChatGLM3-6B**：
    *   进一步提升了基础能力，特别是数学、代码和逻辑推理。
    *   **原生支持工具调用**：模型可以自主决定何时调用外部工具（如计算器、搜索引擎）。
    *   **多模态支持**：推出了多模态版本，可以看懂图片并进行对话。
    *   **代码解释器**：支持在沙箱环境中执行代码。

*   **GLM-4**：
    *   这是智谱 AI 最新一代的基座大模型，整体性能相比 GLM-3 有大幅提升，据称在多项评测中对标 GPT-4。
    *   它不仅仅是聊天模型，更是一个全能的“智能体”平台，支持 All Tools（自动使用浏览器、代码解释器、绘图工具等），具备长文本（128K/1M）处理能力和多模态能力。

### 4. 应用场景
ChatGLM 目前被广泛应用于各个领域：
*   **智能客服**：自动回答用户问题，7x24小时在线。
*   **知识问答**：基于企业内部文档构建知识库助手（RAG）。
*   **内容创作**：写文案、写代码、写报告。
*   **数据分析**：辅助处理 Excel 数据，生成图表。

总结来说，**ChatGLM 是中文世界里最顶尖的开源/商用大模型系列之一**。它不仅在学术上由顶尖高校支持

✅ 智能体初始化成功
   平台: OPENAI
   模型: gpt-4o-mini
   API地址: https://api.openai.com/v1


GPT-4oMini模型是OpenAI推出的一个较小版本的GPT-4模型，旨在提供更高效的计算和更低的资源消耗。虽然具体的参数和架构信息可能与大型版本有所不同，但它仍然保持了GPT系列模型的核心能力，包括自然语言理解和生成的能力。

GPT-4oMini通常被设计用于需要快速响应和较低延迟的应用场景，例如聊天机器人、实时翻译和文本摘要等。它能够在保证一定生成质量的同时，降低对计算资源的需求，使得更多的开发者和企业能够使用先进的自然语言处理技术。

如果你对GPT-4oMini的具体应用、优缺点或其他相关问题有兴趣，可以进一步询问！

ChatGLM回答: 你好！我是由智谱AI研发的GLM大语言模型，很高兴为你介绍我自己以及ChatGLM系列模型。

**ChatGLM** 是由**清华大学 KEG 实验室**和**智谱 AI**（Zhipu AI）共同研发的、支持中英双语的对话语言模型系列。它基于 GLM（General Language Model）架构，旨在让机器能够像人类一样通过自然语言进行流畅的交互。

以下是关于 ChatGLM 模型的详细介绍，主要包含其架构特点、核心优势以及发展历程：

### 1. 核心架构：GLM (General Language Model)
不同于早期的 BERT（擅长理解但不擅长生成）或 GPT（擅长生成但使用自回归单向注意机制），ChatGLM 采用了 **GLM 架构**。
*   **自回归填空**：GLM 通过一种“自回归填空”的任务进行预训练，这使得它结合了双向编码器（理解能力强）和自回归解码器（生成能力强）的优点。
*   **优化部署**：ChatGLM 针对中文语境进行了深度的优化，并且针对推理速度和显存占用进行了特别的工程优化。

### 2. 核心优势
*   **出色的中文能力**：相比许多直接翻译中文提示词的国外模型，ChatGLM 在中文语义理解、成语运用、文化背景理解上表现更加地道和流畅。
*   **低部署门槛（消费级显卡可用）**：这是 ChatGLM 一开始就引爆开源社区的重要原因。通过 INT4 量化技术，ChatGLM-6B 版本可以在仅有 6GB 显存的消费级显卡（如家用游戏显卡）上流畅运行，极大地降低了个人开发者和中小企业使用大模型的门槛。
*   **开源生态**：智谱 AI 开放了多个版本的模型权重，极大地促进了中文大模型社区的发展，衍生出了许多微调版本和应用。
*   **长上下文与工具调用**：随着版本迭代，ChatGLM 系列在长文本处理、代码生成、网络搜索工具调用以及多模态（看图）能力上都有了显著提升。

### 3. 发展历程（主要版本）
ChatGLM 系列迭代非常迅速，以下是几个关键节点：

*   **ChatGLM-6B (初代)**：
    *   于 2022 年合作推出，2023 年开源。
    *   60 亿参数，针对中文问答和对话进行了优化。
    *   特点是“平民化”

---

## 5. 代码核心要点总结

### 5.1 配置文件管理（JSON格式）

**配置文件位置**: `HandsOn/config.json`（不会被Git跟踪）

**配置文件结构** (`HandsOn/config.json`):

```json
{
  "openai": {
    "api_key": "your-openai-api-key-here",
    "base_url": "https://api.openai.com/v1",
    "model": "gpt-4"
  },
  "chatglm": {
    "api_key": "your-chatglm-api-key-here",
    "base_url": "https://open.bigmodel.cn/api/coding/paas/v4",
    "model": "GLM-4.7"
  }
}
```

**安全措施**:
- ✅ `HandsOn/config.json` 已添加到 `.gitignore`，**不会被上传到GitHub**
- ✅ `HandsOn/config.json.example` 作为模板文件，可以提交到Git
- ✅ 使用 `load_config()` 函数统一管理配置加载
- ✅ 配置文件路径自动查找，优先从 `HandsOn/` 目录加载

### 5.2 关键配置参数

| 参数 | 说明 | OpenAI 示例 | ChatGLM 示例 |
|------|------|-------------|--------------|
| `api_key` | API 密钥 | `sk-proj-xxx...` | `xxx.drd6kd...` |
| `base_url` | API 接口地址 | `https://api.openai.com/v1` | `https://open.bigmodel.cn/api/coding/paas/v4` |
| `model` | 模型名称 | `gpt-4` / `gpt-3.5-turbo` | `GLM-4.7` |
| `platform` | 平台标识 | `"openai"` | `"chatglm"` |

### 5.3 核心代码结构

```python
# 1. 加载配置文件
config = load_config("HandsOn/config.json")
# 或自动查找（推荐）
config = load_config()  # 自动查找 HandsOn/config.json

# 2. 创建智能体（从配置文件读取）
agent = QAgent(platform="openai", config=config)
# 或使用 ChatGLM
agent = QAgent(platform="chatglm", config=config)

# 3. 使用智能体
answer = agent.ask("问题")
```

### 5.4 配置文件读取最佳实践

```python
# ✅ 推荐方式：使用配置文件（自动查找）
config = load_config()  # 自动查找 HandsOn/config.json
agent = QAgent(platform="openai", config=config)

# ✅ 使用 ChatGLM 平台
agent = QAgent(platform="chatglm", config=config)

# ✅ 覆盖配置：直接传参（优先级更高）
agent = QAgent(
    platform="openai", 
    config=config,
    model="gpt-3.5-turbo"  # 覆盖config中的model
)

# ❌ 不推荐：硬编码密钥（不安全）
# agent = QAgent(api_key="sk-xxx...", base_url="...", model="...")
```

### 5.5 重要注意事项

1. **API Key 安全** ⚠️
   - ❌ **绝对不要**将 `HandsOn/config.json` 提交到Git
   - ✅ 使用 `HandsOn/config.json.example` 作为模板
   - ✅ 首次使用时：复制示例文件并填写真实密钥
   - ✅ 确保 `.gitignore` 中包含 `HandsOn/config.json`

2. **配置文件格式**
   - 使用标准 JSON 格式
   - 每个平台包含：`api_key`、`base_url`、`model` 三个字段
   - 支持多个平台并存，通过 `platform` 参数切换

3. **错误处理**
   - `load_config()` 自动处理文件不存在或JSON格式错误
   - `QAgent` 初始化时检查必要参数，给出清晰错误提示
   - API 调用失败时返回错误信息而非抛出异常

4. **灵活性设计**
   - 优先使用直接提供的参数（如果提供）
   - 否则从 config 中读取对应平台配置
   - 支持运行时通过 `update_system_prompt()` 切换角色

### 5.6 常见问题排查

| 问题 | 可能原因 | 解决方案 |
|------|---------|---------|
| `FileNotFoundError` | `HandsOn/config.json` 不存在 | 从 `config.json.example` 复制并重命名 |
| `JSONDecodeError` | JSON格式错误 | 检查JSON语法，确保引号、逗号、括号匹配 |
| `KeyError` | 平台名称不存在 | 检查 `platform` 参数（"openai" 或 "chatglm"） |
| API认证失败 | API Key错误或过期 | 检查 `config.json` 中的密钥是否正确 |
| 连接超时 | base_url 格式错误或网络问题 | 检查各平台的正确API地址 |
| 模型不存在 | model 名称错误 | 参考平台文档确认正确的模型名称 |

### 5.7 当前支持的API平台

**配置文件中的平台标识**：

- **OpenAI**: 
  - Platform: `"openai"`
  - Base URL: `https://api.openai.com/v1`
  - 模型示例: `gpt-4`, `gpt-3.5-turbo`

- **ChatGLM**: 
  - Platform: `"chatglm"`
  - Base URL: `https://open.bigmodel.cn/api/coding/paas/v4`
  - 模型示例: `GLM-4.7`

> 💡 **提示**：只要平台支持 OpenAI 兼容的 API 接口格式，都可以使用相同的方式配置和使用。只需在 `config.json` 中添加对应平台的配置即可。